# Binary Bird Classifer from Using CNN

Open Soundscape: http://opensoundscape.org/en/latest/train.html

First, use the following packages to create a machine learning classifier. First, from OpenSoundscape import the following three functions (`run_command`, `binary_train_valid_split`, and `train`) and three classes (`Audio`, `Spectrogram`, and `SingleTargetAudioDataset`).

In [1]:
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from opensoundscape.datasets import SingleTargetAudioDataset

from opensoundscape.helpers import run_command
from opensoundscape.data_selection import binary_train_valid_split
from opensoundscape.torch.train import train

# For working with dataframes, arrays, and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For importing data / audio-related python library
import librosa
import os
import os.path
import pandas as pd



# Split the data into n-segment chunks

We want to first move the files that are under our desired sample rate now 22050Hz (not 44.1kHz) into another directory. Note that our current downsampling method (librosa.load) is not able to upsample our data.

In [35]:
original_dir = '/media/e4e/Rainforest_Data1/'
audioset_dir = '/media/e4e/Rainforest_Data1/audioset_nonbird'
xenocanto_dir = '/media/e4e/Rainforest_Data1/Madre_de_Dios_Xeno_Canto_Birdcalls'

under_44100_xenocanto_dir = os.path.join(original_dir, 'Madre_de_Dios_Xeno_Canto_Birdcalls_under44100')
os.mkdir(under_44100_xenocanto_dir)
under_44100_audioset_dir = os.path.join(original_dir, 'audioset_under44100') 
os.mkdir(under_44100_audioset_dir)

In [36]:
import shutil

def move_files_under_sr(directory, unsampled_dir):
    for file in os.listdir(directory): 
        path_to_file = os.path.join(directory,file)
        data, sample_rate = librosa.load(path_to_file)
        if sample_rate < 44100: 
            print("File is under 44100Hz, will move to a new directory")
            print('file:', file)
            print(path_to_file)
            print(unsampled_dir)
            shutil.move(path_to_file, unsampled_dir)
        else: 
            pass

In [37]:
print(under_44100_xenocanto_dir)
print(under_44100_audioset_dir)

/media/e4e/Rainforest_Data1/Madre_de_Dios_Xeno_Canto_Birdcalls_under44100
/media/e4e/Rainforest_Data1/audioset_under44100


In [39]:
move_files_under_sr(audioset_dir, under_44100_audioset_dir)
move_files_under_sr(xenocanto_dir, under_44100_xenocanto_dir)



File is under 44100Hz, will move to a new directory
file: it8_3gt0uUo_2.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/it8_3gt0uUo_2.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0-Hzui4Qpl4_40.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0-Hzui4Qpl4_40.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0-N62G9etNE_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0-N62G9etNE_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0-yskgO46Bg_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0-yskgO46Bg_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 00cREXaIlnQ_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/00cREXaIlnQ_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 441

File is under 44100Hz, will move to a new directory
file: 0fwt2zSftIc_62.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0fwt2zSftIc_62.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0fZKF6FprcA_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0fZKF6FprcA_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0g-KosNgMYA_70.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0g-KosNgMYA_70.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0Ga7T-2e490_17.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0Ga7T-2e490_17.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0GhmrpFuRNQ_540.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0GhmrpFuRNQ_540.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under

File is under 44100Hz, will move to a new directory
file: 0NZLG7HMiLg_590.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0NZLG7HMiLg_590.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0OhqyNGlJYE_310.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0OhqyNGlJYE_310.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0oI0BvmTLvA_20.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0oI0BvmTLvA_20.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0p7J0O8ew0k_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0p7J0O8ew0k_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 0pLgOm0xgjY_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0pLgOm0xgjY_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is und

File is under 44100Hz, will move to a new directory
file: 0_O6nVfnCH8_210.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/0_O6nVfnCH8_210.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 1-WyAvPxqDM_0.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1-WyAvPxqDM_0.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 1-X6BOetQ5Q_330.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1-X6BOetQ5Q_330.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 10aF24rMeu0_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/10aF24rMeu0_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 10lWpHyN0Ok_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/10lWpHyN0Ok_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under

File is under 44100Hz, will move to a new directory
file: 1OU19ANx4fg_6.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1OU19ANx4fg_6.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 1P5FFxXLSpY_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1P5FFxXLSpY_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 1p9rsIR1ezk_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1p9rsIR1ezk_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 1PPboJuO1Ks_21.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1PPboJuO1Ks_21.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: 1pViEqMXJH0_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/1pViEqMXJH0_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 441

...

File is under 44100Hz, will move to a new directory
file: Gcby1sXAo64_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/Gcby1sXAo64_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: gCjsKL1Mqjk_0.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/gCjsKL1Mqjk_0.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: GCVw4MarbJE_470.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/GCVw4MarbJE_470.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: GCWCB_3fvkI_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/GCWCB_3fvkI_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: LAeWwMC2EaI_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/LAeWwMC2EaI_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 4

File is under 44100Hz, will move to a new directory
file: fKI3LPnPW84_70.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/fKI3LPnPW84_70.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: Fo_FDiZhzDo_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/Fo_FDiZhzDo_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: FT81ZQynj8w_21.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/FT81ZQynj8w_21.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: fVfkPyZl6o0_270.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/fVfkPyZl6o0_270.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: Fxl4CpEjcQU_8.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/Fxl4CpEjcQU_8.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 4

File is under 44100Hz, will move to a new directory
file: lO1rUT_MTjk_210.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/lO1rUT_MTjk_210.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: lo3yvn1av2E_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/lo3yvn1av2E_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: LOHw8cZZ9-U_450.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/LOHw8cZZ9-U_450.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: LOK3eUPj6c4_70.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/LOK3eUPj6c4_70.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: loKkvegkPJ8_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/loKkvegkPJ8_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is und

File is under 44100Hz, will move to a new directory
file: m98BKPAD-84_20.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/m98BKPAD-84_20.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: m9MQdg0k1t0_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/m9MQdg0k1t0_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: Ma0Ye2xMqXA_30.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/Ma0Ye2xMqXA_30.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: Ma19m133Lmk_100.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/Ma19m133Lmk_100.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under 44100Hz, will move to a new directory
file: MA2MTloAjr8_10.000.wav
/media/e4e/Rainforest_Data1/audioset_nonbird/MA2MTloAjr8_10.000.wav
/media/e4e/Rainforest_Data1/audioset_under44100
File is under

KeyboardInterrupt: 

Ended up deciding to downsample to 22050Hz for now. Now, I must move the files in 'audioset_under44100' to 'audioset_nonbird' directory.

In [41]:
for file in os.listdir(under_44100_audioset_dir): 
    path_to_file = os.path.join(under_44100_audioset_dir,file)
    shutil.move(path_to_file, audioset_dir)

## Import Data
Import all the audio files and resample to 22050Hz (not 44.1kHz) and then save each 5 second clip to another directory.
Make sure to make a directory manually of where you would like your clipped files to go to. 

In [54]:
split_path_audioset = os.path.join('/media/e4e/Rainforest_Data1/','audioset_nonbird_split')
#os.mkdir(split_path_audioset)

if not split_path_audioset.exists(): split_path_audioset.mkdir()

In [155]:
split_path_xenocanto = os.path.join('/media/e4e/Rainforest_Data1/', 'Madre_de_Dios_Xeno_Canto_Birdcalls_Split')
os.mkdir(split_path_xenocanto)

#if not split_path_xenocanto.exists(): split_path_xenocanto.mkdir()

In [156]:
def find_n_second_multiples_and_filenames(dir_path, des_dir_path, n): 
    """ 
    dir_path (path): path of a directory with files you want to separate into 5 second clips
    des_dir_path (path): path of a directroy 
    n (int): split data into n-second clips
    wav_data (n dimensional list)
    """
    
    clip_wav_data = []
    allfilenames = []
    timestamp = []

    for filename in os.listdir(dir_path):
        print(filename)
        filename_head = os.path.splitext(filename)[0]
        filename_tail = os.path.splitext(filename)[1]
        filepath = os.path.join(dir_path, filename)
        
        # issue is here with resampling
        data, sr = librosa.load(filepath, sr=22050)

        num_samples_in_n_seconds =  n  * sr 
        length_data = len(data)
        length_in_seconds = length_data / sr 

        if length_data < num_samples_in_n_seconds: 
            pass

        elif length_data > num_samples_in_n_seconds: 
            data = data[:-(length_data % num_samples_in_n_seconds)]
            num_of_n_clips = int(np.floor(length_data / num_samples_in_n_seconds))

            for i in range(num_of_n_clips): 
                start = (i*num_samples_in_n_seconds)/sr
                end = ((i+1)*num_samples_in_n_seconds)/sr

                data_n_seconds = data[i*num_samples_in_n_seconds:(i+1)*num_samples_in_n_seconds]
                timestamp.append([i*n,(i+1)*n])

                # filename 
                new_filename = filename_head + '_%d%s_%d%s' % (start,'s',end,'s') + filename_tail
                allfilenames.append(new_filename)
                
                new_directory = os.path.join(des_dir_path, new_filename)

                # write file
                librosa.output.write_wav(new_directory, data_n_seconds, sr)

        else: #length_data == num_samples_in_n_seconds: 
            allfilenames.append(filename)

            new_directory = os.path.join(des_dir_path, filename)
            librosa.output.write_wav(new_directory, data, sr)
            
    return allfilenames, timestamp


Spltting the data from the Audioset and Xeno Canto data.

In [56]:
original_audioset_path = '/media/e4e/Rainforest_Data1/audioset_nonbird'
allfilenames_audioset, timestamp_audioset = find_n_second_multiples_and_filenames(original_audioset_path, split_path_audioset, 5)

ItjJk-56bF4_0.000.wav
06Nq51sogmQ_10.000.wav
06si40RVDco_30.000.wav
076qdKSC8nU_30.000.wav
07mCK08xGRc_10.000.wav
07rwFurzpw8_21.000.wav
08mf5GxT820_30.000.wav
08QQAMaw24E_30.000.wav
08Ua8a01P5g_13.000.wav
097K1uMIPyQ_30.000.wav
0a5KHX_odJ8_30.000.wav
0AA0R8gbxcI_30.000.wav
0aaLWD7TUSU_30.000.wav
0AkRWunFZPk_190.000.wav
0ARh-9Amrv0_260.000.wav
0BCXcbaLeoA_30.000.wav
0BgI0XxW0kE_340.000.wav
0bKpv6YBl7g_50.000.wav
0Cl6hXpryhY_30.000.wav
0crG6jzAUpE_120.000.wav
0CzC0-cXe8M_30.000.wav
0cZQm65sZjc_50.000.wav
0D9ZiiYArKw_30.000.wav
0dgl6Rvt3X0_30.000.wav
0dVmGVa3Rbw_20.000.wav
0EI7FxbdsUE_410.000.wav
0EiQyHjoaz8_14.000.wav
0EyIdGgbPEo_30.000.wav
0fD-eSRrybM_30.000.wav
0FjrveIENzI_20.000.wav
0fvq8YY28qI_130.000.wav
0fwt2zSftIc_62.000.wav
0fZKF6FprcA_30.000.wav
0g-KosNgMYA_70.000.wav
0Ga7T-2e490_17.000.wav
0GhmrpFuRNQ_540.000.wav
0H25ve3qts4_1.000.wav
0hjqlmdPT6g_50.000.wav
0hOD_iIeePk_30.000.wav
0hUft9scxzY_460.000.wav
0HzbpwB3xDk_30.000.wav
0IMxRf5KZBY_30.000.wav
0IwGsEzITQs_20.000.wav
0IxyU

3hb1pYdEuLE_6.000.wav
3hLxPQpmfQo_30.000.wav
3InTvnjfheM_10.000.wav
3iQ_xRurgS8_30.000.wav
3IX29m1L9UY_30.000.wav
3Jc3uW-Lcs0_30.000.wav
3juGfzF29Sc_140.000.wav
3kXukXBvDQQ_220.000.wav
3Liy9uBgsQM_60.000.wav
3LLKsRSkAbE_110.000.wav
3LtXT2KVI-c_30.000.wav
3LxtO_vMokk_10.000.wav
3lxTtZfYS18_30.000.wav
3m2vl8_-ESA_40.000.wav
3M9qo2FUqKg_110.000.wav
3mDPQ_CPopw_30.000.wav
3mkiTkMVemM_30.000.wav
3Mo-YFd31rs_16.000.wav
3mtdpXbogzk_180.000.wav
3MUeg3nD2OU_120.000.wav
2zKyMjBnL0Y_60.000.wav
2zO-nlygkFo_30.000.wav
2__RQYoZrjI_19.000.wav
3-a8i_MEUl8_30.000.wav
3-QgZPCM4nk_160.000.wav
302w8uJn_zE_30.000.wav
30HtOudq2v8_20.000.wav
31Ij3GJLGjI_30.000.wav
31tnXPlhA7w_30.000.wav
32Wp1Qo1MaY_13.000.wav
33fqw0k_qkM_220.000.wav
33UghN1hNF8_4.000.wav
35CqIfFXjyQ_21.000.wav
35HFjLYQiwY_30.000.wav
35ig21J679o_30.000.wav
36YUxW6ImW4_0.000.wav
37ER1zpg_F4_30.000.wav
37QWwibfvXU_30.000.wav
38F6eeIR-s0_30.000.wav
38Z6D7yfsA4_30.000.wav
39IZIbmQPzA_40.000.wav
39rXWSXGOyo_80.000.wav
3AcKnYk_cLY_30.000.wav
3aM1Kt

6dgTnGDYiZc_9.000.wav
6iNLmtQmy3Y_30.000.wav
6o-kOtY8tdY_250.000.wav
6vCO2y-bYoU_30.000.wav
6OgSNQOTw2U_60.000.wav
6om-fZvxe70_30.000.wav
6PLlWvC4BiQ_160.000.wav
6PPu1C_00po_30.000.wav
6psjy93yoTA_30.000.wav
6q0MGwl-xZA_30.000.wav
6qrJMLyyHII_440.000.wav
6qtIvxTeL1c_30.000.wav
6r00ZvsYi9g_150.000.wav
6R8cO4ARzkY_30.000.wav
6sfjS8pGgjM_30.000.wav
6slrju_ar9U_280.000.wav
6SNm9kiw9Dk_30.000.wav
6U942AYlcXA_30.000.wav
6UY-HKyolKk_30.000.wav
6VBURvUgzls_30.000.wav
7S88FsFE5EE_30.000.wav
7sgNkeZCPQc_30.000.wav
7sjPS_ru0Kw_30.000.wav
7skzapW_Jy4_24.000.wav
7ssccf2qjx8_30.000.wav
7S_iHrdfrvQ_30.000.wav
7T04388Ijk8_30.000.wav
7t2HZLZ1PQs_28.000.wav
7tbxN_Mq8Qg_270.000.wav
7tfaSPv7ccU_80.000.wav
7TGF85i5nC0_10.000.wav
7TkvzwbNYwU_60.000.wav
7TTnUHzI8NQ_350.000.wav
7u4X2wK1SlY_530.000.wav
7uJZ85F8HrU_450.000.wav
58VP5N8owns_150.000.wav
5Ad0cKvD7sY_230.000.wav
5AdvAAdZ8Xs_570.000.wav
5AmicEkp3Oc_170.000.wav
5asfoHsxI2w_30.000.wav
5b6lStP8UXE_380.000.wav
5B7d8e42ODw_30.000.wav
5BLIgNC4-4w_160.000.w

84E9i9_ELBs_30.000.wav
84fDGWoRtsU_210.000.wav
84flVacRHUI_21.000.wav
84jS0NjXLGc_30.000.wav
84vLeRd4pw0_480.000.wav
86p6UVFIcFI_30.000.wav
87NvADAjdYo_51.000.wav
88N23-hjddY_280.000.wav
88OXNd0yTMI_40.000.wav
89t0486uIfo_90.000.wav
8a0Ctyt3Fpo_270.000.wav
8A1xDCkq5dk_30.000.wav
8AY0aXAQjwY_250.000.wav
8aYqigfxXZE_80.000.wav
8AYXpxT4jcs_90.000.wav
8bakI0ITCqQ_30.000.wav
9VziOIkNXsE_30.000.wav
9w9j2IPMtu8_270.000.wav
9wkgTdE7nz8_30.000.wav
9wYmnN99riA_50.000.wav
9XBiG9vs-Us_480.000.wav
9xLXQP-tulE_150.000.wav
9xm0YLR853E_30.000.wav
9xvmNaF73fo_30.000.wav
9Ycf6vsdAhk_30.000.wav
9ye2Fn62xDc_60.000.wav
9yZ1v9fFct8_28.000.wav
9zfId46t1vM_30.000.wav
9zW-E0XdWdw_160.000.wav
9ZYj5toEbGA_0.000.wav
9_eXYyA8N18_530.000.wav
9_L9CXHPd9M_100.000.wav
a-NddvpUTi8_8.000.wav
A-oSBMP-Zy4_30.000.wav
a-RitCsJEJw_10.000.wav
a0fymQ8ppG0_150.000.wav
A0J_487VquE_30.000.wav
a18gR7Jc7s4_440.000.wav
a1tG1JLWeVY_14.000.wav
A1TNpj9hHW0_30.000.wav
a2Iik1XpkDk_25.000.wav
A3a5-aOyIJk_30.000.wav
a3jZ7QBDhgc_30.000.wav


bY1f3qXSZa0_510.000.wav
ByacfcUJSYA_30.000.wav
BYBRY2mupcc_30.000.wav
BymkWsAcLR4_0.000.wav
Byndzwec79o_30.000.wav
ByNI_BUprIU_40.000.wav
bYnO0qJZ_XA_240.000.wav
bYwoYjbPm-I_30.000.wav
bZadyuv2utE_80.000.wav
BzQw9qtl32o_10.000.wav
Bzr_Akg7WJ0_30.000.wav
afsW2VeCfak_10.000.wav
AFsX8-IXV-0_30.000.wav
AFwgR9v62DY_30.000.wav
AFwtBviVhhM_30.000.wav
AFz5TIs_Gug_30.000.wav
AG105FogvfY_400.000.wav
aG5AePYS13k_20.000.wav
AG7Zm7D4JIE_30.000.wav
AGA3hiGZJCo_0.000.wav
AGNqX_OL-dU_60.000.wav
Agoupe7nOhk_190.000.wav
AGqCcrX619E_60.000.wav
AHm_49I6rzM_120.000.wav
AHom7lBbtoY_30.000.wav
AHoWincIkzc_30.000.wav
AHPRhJ4u0ss_27.000.wav
AIDJFkDURPY_30.000.wav
AiHZ9hODcXg_450.000.wav
AiLw2qlOGhY_30.000.wav
AIuayXjDos8_130.000.wav
AiV8hsnubyA_30.000.wav
AIVt3e5EVtc_70.000.wav
aIXGe5fsx7A_0.000.wav
aiyBTv4PKcM_60.000.wav
AIz2rceSCtQ_30.000.wav
aj1pv3GKtZY_60.000.wav
BZU01hhyVKs_480.000.wav
bZuXMxR2S4U_40.000.wav
bZw3W8YOk54_30.000.wav
b_kvMCmoZw0_30.000.wav
B_nDJh0ASP4_30.000.wav
b_RtXz277T0_30.000.wav
c-8SLU

...

wdFUIHCht9Q_90.000.wav
wDJufoP1hLQ_20.000.wav
WDoZneKkmVg_30.000.wav
wdPWomHjCx8_150.000.wav
We-bFXFjWaM_30.000.wav
Of8y_rXXeSw_30.000.wav
OfMZNT0DJZI_30.000.wav
OFSR16VOiak_330.000.wav
OFWD8T1wqHo_60.000.wav
Og6_smbvC4g_30.000.wav
OgBjjkGNjHs_0.000.wav
ogyY4T6Y0rQ_0.000.wav
oH-u_pxYdF0_130.000.wav
OH8urnIthoQ_60.000.wav
oHa_rQrbTEA_110.000.wav
oHGNVbNyndk_110.000.wav
oHKTmTLEy68_11.000.wav
oHp-J8QgAd8_30.000.wav
OIESgZwkkpQ_30.000.wav
oiLHBkHgkAo_0.000.wav
_SWnLeXatrw_430.000.wav
_tDF9h1Y5o4_400.000.wav
_TniryQZy0s_320.000.wav
_TUZiiMy1iI_30.000.wav
_U8kAFAm8tQ_30.000.wav
_uFOeFc7h28_20.000.wav
_UmY2aYt1YQ_30.000.wav
_uNDm6YfN2k_90.000.wav
_VdCrHztInA_400.000.wav
_vf2o95UdCk_10.000.wav
_vudBMe0eDo_200.000.wav
_W0goL0IzmE_22.000.wav
_WCpjcXIYzA_10.000.wav
_WepUW63bsQ_30.000.wav
_WfAopep2qQ_30.000.wav
_WGTedHMJDA_380.000.wav
_WHA4Awjq1Q_200.000.wav
_WheLp0RdLQ_30.000.wav
_WLJGMQYWWo_30.000.wav
_W_Mc16Gi4Q_30.000.wav
_xMMWHVbjGk_30.000.wav
_xuq9rBndUE_0.000.wav
_XYa-N7WrTQ_30.000.wav
_yr

xvanhCQ9jho_12.000.wav
xvAQ44hx3_k_220.000.wav
xVaz2kQsKmg_170.000.wav
xVchMQHrQLk_30.000.wav
xvdgRGPcMpw_30.000.wav
XvkNRGfbdw8_17.000.wav
XvLGAa1TS8U_30.000.wav
xvw3Z5-6h1A_10.000.wav
xVylfJbxcm0_70.000.wav
XW8pSKLyr0o_20.000.wav
xWpmJvOkAyY_530.000.wav
xwQE_69dbgQ_30.000.wav
XWTSXTZLYHg_30.000.wav
XwTzgkH2j_s_30.000.wav
xWvzWiD3Wmo_60.000.wav
xxCnmao8FAs_20.000.wav
XXfaG4T1eB0_30.000.wav
XXqu6jihhvs_40.000.wav
XxUxOzujJAg_30.000.wav
xY4YrFk_PI8_570.000.wav
XYBnvCphgtw_290.000.wav
XyIAPpcvVro_0.000.wav
XYLDjkXAuec_380.000.wav
xyrOTEmrFt8_30.000.wav
XyT6JsD0o88_20.000.wav
XYuNLOzROLg_7.000.wav
XZ5oHxjfHKY_30.000.wav
XZhLMTOmM9w_30.000.wav
xZlTtt3gvXk_5.000.wav
XZPIJ7x098g_50.000.wav
XzRoWpGfCFE_410.000.wav
yd86MVEWjK0_120.000.wav
YdiM5lFJWSA_40.000.wav
ydIzbRlPs7s_100.000.wav
ydRWFW8Svrs_30.000.wav
YdXQJS3Yv0Y_30.000.wav
YE2rN3xknlk_30.000.wav
yEahPgS0Kes_30.000.wav
yf1R8QkpGpo_23.000.wav
yf2rGrT_yyY_10.000.wav
YfeXdgLNp8E_160.000.wav
YFFghjya8vk_30.000.wav
YFHYP_TSWII_30.000.wav
yFWW

ZWEPbCFmzOg_230.000.wav
ZWJqgdi1yU4_100.000.wav
ZwUTUKXXlto_50.000.wav
rgcDePlYpCM_420.000.wav
rgCyK-VTLOM_30.000.wav
rhFGGwRArL4_100.000.wav
rhWrIKdn8ic_116.000.wav
RhzsxX39VMU_30.000.wav
RI2T1BBL_8I_30.000.wav
rirmqX7pIH0_280.000.wav
RjF9D8xDYFg_30.000.wav
RjFFqAjF12c_20.000.wav
Rjlj0frH8XA_250.000.wav
rjnUJ4AzhgQ_240.000.wav
RJOBq50THOk_30.000.wav
rJZgUpzqAyY_70.000.wav
RjzKZrdgmlA_30.000.wav
RJzVY-srLtI_60.000.wav
ZXbnWlzc0ro_540.000.wav
ZXCPSAPFU_0_490.000.wav
zXFhweU3fCo_300.000.wav
zxjDTJ_DA0s_150.000.wav
zxNUS4T4Zsc_80.000.wav
Zxq2_xOLT8o_0.000.wav
Zxrd4UgvZhA_23.000.wav
Zy0VM2u3nJI_0.000.wav
zY2CAiwNJ6A_30.000.wav
ZyiDGq0qh2s_30.000.wav
ZylkgLeV-bU_23.000.wav
ZyvTbfk7omw_0.000.wav
zz35Va7tYmA_30.000.wav
ZZ5IinzbauY_30.000.wav
ZZ6GRocWW38_90.000.wav
zZ7fLYS-CE8_10.000.wav
zzbTaK7CXJY_30.000.wav
ZZcwENgmOL0_280.000.wav
ZZDvShjLnAE_30.000.wav
zZi-X9M0G5w_30.000.wav
Ry5c8frvq1g_390.000.wav
RybkZKoNc0s_30.000.wav
rYgjgcU8dJQ_260.000.wav
rYJLMgfQIMg_10.000.wav
ryqrfRxqYfo_60.000.wav

Vby0FZ3eIlA_30.000.wav
Vf8FlhjEAro_0.000.wav
VgvVPVMhRGQ_350.000.wav
VjjS4B1FMNM_180.000.wav
VoAfb3f04mo_30.000.wav
Vr_vFgV6kdw_300.000.wav
vuA8DmYCJb0_180.000.wav
Vx6mttDHWfo_30.000.wav
vsBI27QYYhw_34.000.wav
VSbuen0NWaw_200.000.wav
VsdOoLvOZQ8_17.000.wav
vsDTdpTIIMM_70.000.wav
VSf-6xznJyk_240.000.wav
VSkdiYt87rg_30.000.wav
VsLgHTWOo6k_30.000.wav
vsM67sj0Joc_60.000.wav
vSW31TUJLi4_60.000.wav
vSwxayAGuaM_30.000.wav
VswY2mI7Wbo_360.000.wav
vTCzh-OczyY_30.000.wav
vtE1J8HsCUs_30.000.wav
VtF461zWIkk_30.000.wav
VTI1k--NxMk_30.000.wav
VTqMAfUWOk0_190.000.wav
vtVfl5Ff5lw_30.000.wav
VTwVF3xRSWg_12.000.wav
Vu-x88iPIv8_13.000.wav
vuAcrDFvApA_40.000.wav
vuD0qyibQyQ_30.000.wav
VUGB-2HL6YE_12.000.wav
vuGfxAyFnSA_30.000.wav
VugK05JE7EA_400.000.wav
VUIQZEC_VFo_30.000.wav
VUosI4YQook_30.000.wav
Vuqt6ip9kME_30.000.wav
VV-DgnPPhSo_220.000.wav
Vv0rVOqKhkQ_40.000.wav
vv8I11kdDzo_0.000.wav
vVCnBdAK678_10.000.wav
VViXFIbJNbk_130.000.wav
VVj6ISLoddo_30.000.wav
vvJKJH84cQ0_30.000.wav
vVo3xT3cHBA_50.000.wav
vV

Zqrd8cZ7UTw_550.000.wav
ZTGysFKGPcc_30.000.wav
zx43ZN4pQNw_30.000.wav
z_Tx6JrUb80_240.000.wav
_8MvhMlbwiE_40.000.wav
_jZha6gjkK8_14.000.wav
_oiwqeGo5-Y_10.000.wav
_sRksbqOLPM_1.000.wav


In [157]:
original_xenocanto_path = '/media/e4e/Rainforest_Data1/Madre_de_Dios_Xeno_Canto_Birdcalls'
allfilenames_bird, timestamp_bird = find_n_second_multiples_and_filenames(original_xenocanto_path, split_path_xenocanto, 5)

XC47568 - White-winged Becard - Pachyramphus polychopterus nigriventris.mp3
XC146509 - Three-striped Flycatcher - Conopias trivirgatus.mp3
XC146524 - Black-faced Cotinga - Conioptilon mcilhennyi.mp3
XC146525 - Olive Oropendola - Psarocolius bifasciatus yuracares.mp3
XC146526 - Mato Grosso Swift - Chaetura viridipennis.mp3
XC146527 - Buff-throated Saltator - Saltator maximus.mp3
XC146528 - Buff-throated Saltator - Saltator maximus.mp3
XC146529 - Yellow-browed Sparrow - Ammodramus aurifrons.mp3
XC146530 - Tui Parakeet - Brotogeris sanctithomae.mp3
XC146531 - Black-faced Cotinga - Conioptilon mcilhennyi.mp3
XC146532 - Tui Parakeet - Brotogeris sanctithomae.mp3
XC146534 - Greyish Saltator - Saltator coerulescens.mp3
XC146535 - Greyish Saltator - Saltator coerulescens.mp3
XC146536 - Spot-breasted Woodpecker - Colaptes punctigula.mp3
XC146538 - Blue-black Grassquit - Volatinia jacarina.mp3
XC146539 - Chestnut-bellied Seedeater - Sporophila castaneiventris.mp3
XC146540 - Blue-black Grassquit 

XC91336 - Greyish Saltator - Saltator coerulescens.mp3
XC91337 - Paint-billed Crake - Neocrex erythrops.mp3
XC91339 - Double-collared Seedeater - Sporophila caerulescens.mp3
XC91438 - Pale-rumped Swift - Chaetura egregia.mp3
XC91439 - Black-tailed Tityra - Tityra cayana.mp3
XC91440 - Yellow-rumped Cacique - Cacicus cela.mp3
XC91441 - Yellow-rumped Cacique - Cacicus cela.mp3
XC91442 - Yellow-browed Sparrow - Ammodramus aurifrons.mp3
XC91443 - Eastern Wood Pewee - Contopus virens.mp3
XC91444 - Little Woodpecker - Veniliornis passerinus.mp3
XC91445 - Ihering's Antwren - Myrmotherula iheringi.mp3
XC91446 - Rusty-belted Tapaculo - Liosceles thoracicus.mp3
XC91448 - Red-necked Woodpecker - Campephilus rubricollis.mp3
XC91449 - Black-faced Cotinga - Conioptilon mcilhennyi.mp3
XC91450 - Plum-throated Cotinga - Cotinga maynana.mp3
XC91451 - Black-faced Cotinga - Conioptilon mcilhennyi.mp3
XC91452 - Black-faced Cotinga - Conioptilon mcilhennyi.mp3
XC91453 - Ihering's Antwren - Myrmotherula iheri

# Labelling Dataset

In order to use a binary bird classifier with a CNN, we need to label the audio files we input to see if it has a bird or not. The dataset we will be using to train the CNN will ultimately have audio files from AudioSet and Xeno Canto data. Let us label the data accurately

In [169]:
print(allfilenames_audioset[0:5])
print(len(allfilenames_audioset))
print(allfilenames_bird[0:5])
print(len(allfilenames_bird))
allfilenames = allfilenames_audioset + allfilenames_bird
assert (len(allfilenames))== len(allfilenames_audioset) + len(allfilenames_bird)

['ItjJk-56bF4_0.000.wav', 'ItjJk-56bF4_0.000.wav', '06Nq51sogmQ_10.000.wav', '06Nq51sogmQ_10.000.wav', '06si40RVDco_30.000.wav']
12269
['XC47568 - White-winged Becard - Pachyramphus polychopterus nigriventris_0s_5s.mp3', 'XC47568 - White-winged Becard - Pachyramphus polychopterus nigriventris_5s_10s.mp3', 'XC47568 - White-winged Becard - Pachyramphus polychopterus nigriventris_10s_15s.mp3', 'XC47568 - White-winged Becard - Pachyramphus polychopterus nigriventris_15s_20s.mp3', 'XC47568 - White-winged Becard - Pachyramphus polychopterus nigriventris_20s_25s.mp3']
2801


Since we are using xeno canto data (which is labelled data - bird and not bird), we need to label each of these files. In order to do so, we extract this information from the filename. Note that all the xeno canto data starts with 'XC'. 

In [180]:
def label_files(file_list): 
    bird_present_binary = []
    bird_species_list = []
    for f in file_list: 
        if f[:2] == 'XC' and os.path.splitext(f)[1]=='.mp3':
            bird_present_binary.append(1)
            bird_species_list.append(os.path.splitext(f.split("-",1)[1].strip())[0])
        else:
            bird_present_binary.append(0)
            bird_species_list.append('N/A')
    return bird_species_list, bird_present_binary

In [181]:
bird_species_list, bird_present_binary = label_files(allfilenames)

print(len(allfilenames))
assert(len(allfilenames)==len(bird_present_binary))
assert(len(bird_species_list)==len(bird_present_binary))

15070


## Put all this data into a Pandas dataframe
There would be a pandas dataframe that would include filenames, the timestamp of the file, if a bird is present in the file, and the bird species.

In [186]:
labels = {'Filenames': allfilenames,
        'Bird present': bird_present_binary,
        'Bird species': bird_species_list
        }

df = pd.DataFrame(labels, columns = ['Filenames', 'Bird present', 'Bird species'])
label_dict = {0:'bird-absent', 1:'bird-present'}

print(df.head(10))
print(df.tail(10))
print(label_dict)

                Filenames  Bird present Bird species
0   ItjJk-56bF4_0.000.wav             0          N/A
1   ItjJk-56bF4_0.000.wav             0          N/A
2  06Nq51sogmQ_10.000.wav             0          N/A
3  06Nq51sogmQ_10.000.wav             0          N/A
4  06si40RVDco_30.000.wav             0          N/A
5  06si40RVDco_30.000.wav             0          N/A
6  076qdKSC8nU_30.000.wav             0          N/A
7  076qdKSC8nU_30.000.wav             0          N/A
8  07mCK08xGRc_10.000.wav             0          N/A
9  07rwFurzpw8_21.000.wav             0          N/A
                                               Filenames  Bird present  \
15060  XC97444 - Acre Tody-Tyrant - Hemitriccus cohnh...             1   
15061  XC97444 - Acre Tody-Tyrant - Hemitriccus cohnh...             1   
15062  XC97444 - Acre Tody-Tyrant - Hemitriccus cohnh...             1   
15063  XC97444 - Acre Tody-Tyrant - Hemitriccus cohnh...             1   
15064  XC97444 - Acre Tody-Tyrant - Hemitriccus

Ensure that the 'Filenames' column in the pandas dataframe has a path to the file. Note that we saved all the files into split files.

In [183]:
print(split_path_audioset)
print(split_path_xenocanto)
print(len(allfilenames))

/media/e4e/Rainforest_Data1/audioset_nonbird_split
/media/e4e/Rainforest_Data1/Madre_de_Dios_Xeno_Canto_Birdcalls_Split
15070


In [ ]:
# Fix issue of labelling from here:

In [188]:
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])
print(df2['a'][0])

1


In [189]:
for i in range(len(allfilenames)): 
    if allfilenames[i][:2] == 'XC' and os.path.splitext(allfilenames[i])[1]=='.mp3':
        df['Filenames'][i] = os.path.join(split_path_xenocanto, allfilenames[i])
    else: 
        df['Filenames'][i] = os.path.join(split_path_audioset, allfilenames[i])

In [190]:
print(df.head(10))
print(df.tail(10))

                                           Filenames  Bird present  \
0  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
1  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
2  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
3  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
4  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
5  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
6  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
7  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
8  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   
9  /media/e4e/Rainforest_Data1/audioset_nonbird_s...             0   

  Bird species  
0          N/A  
1          N/A  
2          N/A  
3          N/A  
4          N/A  
5          N/A  
6          N/A  
7          N/A  
8          N/A  
9          N/A  
                                          

Let us export the dataframe into a csv to ensure that it is labelling the files correctly. 

TODO: figure out the bird species extraction with the new file naming system (whoops)

In [191]:
df.to_csv('dataframe_binary_bird.csv')

# Machine learning: training

Biologists are increasingly using acoustic recorders to study species of interest. Many bioacousticians want to determine the identity of the sounds they have recorded; a variety of manual and automated methods exist for this purpose. Automated methods can make it easier and faster to quickly predict which species or sounds are in one's recordings.

Using a process called machine learning, bioacousticians can create (or "train") algorithms that can predict the identities of species vocalizing in acoustic recordings. These algorithms, called classifiers, typically do not identify sounds using the recording alone. Instead, they use image recognition techniques to identify sounds in spectrograms created from short segments of audio.

This tutorial will guide you through the process of training a simple classifier for a single species. To download the tutorial as a Jupyter Notebook and run it on your own computer, click the "Edit on GitHub" button at the top right of the tutorial. You will have to [install OpenSoundscape](installation.html#installation) to use the tutorial.

Import the following machine learning-related modules. OpenSoundscape uses PyTorch to do machine learning.

In [192]:
import torch
import torch.nn
import torch.optim
import torchvision.models

Lastly, use a few miscellaneous functions.

In [193]:
# For interacting with paths on the filesystem
import os.path
from pathlib import Path
from sklearn.model_selection import train_test_split

# For handling output of the training function
import io
from contextlib import redirect_stdout

## Prepare audio data

### Download labeled audio files

Training a machine learning model requires some pre-labeled data. These data, in the form of audio recordings or spectrograms, are labeled with whether or not they contain the sound of the species of interest. These data can be obtained from online databases such as Xeno-Canto.org, or by labeling one's own ARU data using a program like Cornell's "Raven" sound analysis software.

For ML pipelines, we need a training, validation, and test set. 
Outline: 
- Use audio augmented data of the actual test set for the training and validation set (split)
- Then use the actual test set (collection of Xeno Canto data and Audioset) for the test set 

1. Training set: sample of data used to fit the model AKA the actual dataset that we use to train the model (audio augmented set of data)
2. Validation: sample of data used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters
3. Test set: sample of data used to provide an unbiased evaluation of a final model fit on the training dataset (usually unlabelled data)

Sources: 
https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7 

### Audio Augmentation
Reference for audio augmentation as seen here: https://github.com/UCSD-E4E/passive-acoustic-biodiversity/blob/master/Audio_Data_Augmentation/Data_Augmentation_Tutorial.ipynb

In [194]:
print(len(df))
df.head()

15070


,Filenames,Bird present,Bird species
0,/media/e4e/Rainforest_Data1/audioset_nonbird_s...,0,N/A
1,/media/e4e/Rainforest_Data1/audioset_nonbird_s...,0,N/A
2,/media/e4e/Rainforest_Data1/audioset_nonbird_s...,0,N/A
3,/media/e4e/Rainforest_Data1/audioset_nonbird_s...,0,N/A
4,/media/e4e/Rainforest_Data1/audioset_nonbird_s...,0,N/A


In [195]:
train_df, valid_df = train_test_split(df, train_size=0.8, stratify=labels['Bird present'])


In [196]:
num_samples = df.shape[0]
num_present = sum(df['Bird present'] == 1)
print(f"Fraction of original dataframe with bird species present: {num_present/num_samples:.2f}")

num_train_samples = train_df.shape[0]
num_train_present = sum(train_df['Bird present'] == 1)
print(f"Fraction of train samples with bird species present: {num_train_present/num_train_samples:.2f}")
    
num_valid_samples = valid_df.shape[0]
num_valid_present = sum(valid_df['Bird present'] == 1)
print(f"Fraction of train samples with bird species present: {num_valid_present/num_valid_samples:.2f}")


Fraction of original dataframe with bird species present: 0.19
Fraction of train samples with bird species present: 0.19
Fraction of train samples with bird species present: 0.19


### Format as `SingleTargetAudioDataset`s

Turn these dataframes into "Datasets" using the `SingleTargetAudioDataset` class. Once they are set up in this class, they can be used by the training algorithm. Data augmentation could be applied in this step, but is not demonstrated here; for more information, see the [relevant API documentation](api.html#opensoundscape.datasets.SingleTargetAudioDataset).

To use this class, specify the names of the relevant columns in the dataframes:

In [197]:
train_dataset = SingleTargetAudioDataset(
    df=train_df, label_dict=label_dict, label_column='Bird present', filename_column='Filenames', add_noise=True)
valid_dataset = SingleTargetAudioDataset(
    df=valid_df, label_dict=label_dict, label_column='Bird present', filename_column='Filenames', add_noise=True)


## Train the machine learning model
Next, set up the architecture of the machine learning model and train it. 

### Set up model architecture
The model architecture is a neural network. Neural networks are so-named for their loose similarity to neurons. Each **neuron** takes in a small amount of data, performs a transformation to the data, and passes it on with some weight to the next neuron. Neurons are usually organized in **layers**; each neuron in one layer can be connected to one or multiple neurons in the next layer. Complex structures can arise from this series of connections.

The neural network used here is a combination of a feature extractor and a classifier. The **feature extractor** is a convolutional neural network (CNN). CNNs are a special class of neural network commonly used for image classification. They are able to interpret pixels that are near each other to identify shapes or textures in images, like lines, dots, and edges. During the training process, the CNN learns which shapes and textures are important for distinguishing between different classes.

The specific CNN used here is `resnet18`, using the `pretrained=True` option. This means that the model loaded is a version that somebody has already trained on another image dataset called ImageNet, so it has a head start on understanding features commonly seen in images. Although spectrograms aren't the same type of images as the photographs used in ImageNet, using the pretrained model will allow the model to more quickly adapt to identifying spectrograms.

In [198]:
model = torchvision.models.resnet18(pretrained = True)

Although we refer to the whole neural network as a classifier, the part of the neural network that actually does the species classification is its `fc`, or "fully connected," layers. This part of the neural network is called "fully connected" because it consists of several layers of neurons, where every neuron in each layer is connected to every other neuron in its adjacent layers.

These layers come after the CNN layers, which have already interpreted an image's features. The `fc` layers then use those interpretations to classify the image. The number of output features of the CNN, therefore, is the number of input features of the `fc` layers:

In [199]:
model.fc.in_features

512

Use a `Linear` classifier for the `fc`. To set up the `Linear` classifier, identify the input and output size for this classifier. As described above, the `fc` takes in the outputs of the feature extractor, so `in_features = model.fc.in_features`. The model identifies one species, so it has to be able to output a "present" or "absent" classification. Thus, `out_features=2`. A multi-species model would use `out_features=number_of_species`.

In [200]:
model.fc = torch.nn.Linear(in_features = model.fc.in_features, out_features = 2)


### Train the model

Next, create set up a directory in which to save results.

In [201]:
results_path = Path('/media/e4e/Rainforest_Data1/model_train_results')

if not results_path.exists(): results_path.mkdir()


In [202]:
print(results_path)

/media/e4e/Rainforest_Data1/model_train_results


The scikit-learn function may throw errors when calculating metrics; the following code will silence them.

In [203]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

Finally, run the model training with the following parameters:
* `save_dir`: the directory in which to save results (which is created if it doesn't exist)
* `model`: the model set up in the previous cell
* `train_dataset`: the training dataset created using `SingleTargetAudioDataset`
* `optimizer`: the optimizer to use for training the algorithm
* `loss_fn`: the loss function used to assess the algorithm's performance during training
* `epochs`: the number of times the model will run through the training data
* `log_every`: how frequently to save performance data and save intermediate machine learning weights (`log_every=1` will save every epoch)

The `train` function allows the user to control more parameters, but they are not demonstrated here. For more information, see the [train API](http://opensoundscape.org/en/latest/api.html#module-opensoundscape.torch.train).

In [204]:
train_outputs = io.StringIO()

with redirect_stdout(train_outputs):
    train(
        save_dir = results_path,
        model = model,
        train_dataset = train_dataset,
        valid_dataset = valid_dataset,
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3),
        loss_fn = torch.nn.CrossEntropyLoss(),
        epochs=10,
        log_every=1,
        print_logging=True,
    )
    

FileNotFoundError: [Errno 2] No such file or directory: '/media/e4e/Rainforest_Data1/audioset_nonbird_split/5BLIgNC4-4w_160.000.wav'

## Evaluate model performance
When training is complete, it is important to check the training results to see how well the model identifies sounds. This model was only trained on a limited amount of data, so the model is expected to not be usable--it is for demonstration purposes only.

The outputs of the training function were saved to `train_outputs`. Check out the first 100 characters of this output.

Notes on the statistics: 
- loss: In neural network, we want to minimize the error and the objective function for this is called a loss function. The value calculated by the loss function is referred to as "loss" 
- accuracy: Accuracy is the fraction of prediction our model got right aka the number of correct predictions / total number of predictions
- precision: Answers the question: what proportion of positive identifications was actually correct? A model that produces no false positives (e.g. labels an audio file as a bird but does not contain any bird sounds) would have a precision of 1.0. Equation: TP / (TP + FP) 
- recall: Answers the question: what proportion of actual positives was identified correctly? A model that produces no false negatives has a recall of 1.0. So, in equation form: TP/ TP + FN
- f1: A metric that relies on both precision and recall. 

Resources: 
- https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall 

In [ ]:
source_text = train_outputs.getvalue()
print(source_text)

These functions help to parse the log text. They simply extract the resulting "metric" in each epoch. Metrics include accuracy, precision, recall, and f1 score.

In [ ]:
def extract_all_lines_containing(source_text, str_to_extract):
    """Case-sensitive search for lines containing str_to_extract"""
    finished = False
    lines = source_text.split('\n')
    extract_lines = [line for line in lines if str_to_extract in line]
    return extract_lines

def strip_log(log, sep=':     '):
    return log.split(sep)[1]

def get_metric_from_log(source_text, metric):
    if 'precision' in metric or 'recall' in metric:
        return [float(strip_log(line, sep=': ').strip('[]').split()[1]) for line in extract_all_lines_containing(source_text, metric)]
    return [float(strip_log(line, sep=': ')) for line in extract_all_lines_containing(source_text, metric)]

In [ ]:
# get_metric_from_log(source_text, 'precision')
# get_metric_from_log(source_text, 'accuracy')
# get_metric_from_log(source_text, 'recall')
# get_metric_from_log(source_text, 'f1 score')

Plot the validation accuracy each epoch. These results will look different every time the model is trained, as it is a stochastic process (randomly determined).

In [ ]:
metrics_to_plot = ['valid_accuracy', 'train_accuracy']
fig, ax = plt.subplots(1, 1)
for metric in metrics_to_plot:
    results = get_metric_from_log(source_text, metric)
    ax.scatter(range(len(results)), results)
ax.set_ylim(0, 1)
ax.set_title('model training results')
ax.legend(metrics_to_plot)
plt.show()

Lastly, this command "cleans up" by deleting all the downloaded files and results. Only run this if you are ready to remove the results of this analysis.

In [ ]:
#import shutil
# Delete results
#shutil.rmtree(results_path)